# Sand pit

When I worked on AoC 2015, I noticed I can do better than that. I had no idea what to expect, and my approach was horrible. After reading the assignment, I had to search the internet regarding certain features in python. However, I did not want to just sit down and read all the python documentations from the start to the end. So I got some important highlight of the language to focus on. So, I decided to get some hands on training on the toolset that Peter Norvig <a href='https://nbviewer.org/github/norvig/pytudes/blob/main/ipynb/Advent-2021.ipynb'>used</a>. After, I will solve the puzzles, then compare mine with his and see how horrible mine is.

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

## Annotations

<code>annotations</code> delays evaluation of function annotations. Without the future import, NameError is raised below. However, after the import, it works without any issue.

In [2]:
class A:
    def a(self) -> A:
        return

## Collections

### Counter 

Counter can be used to count things.

There are many ways to initiate the Counter.

In [3]:
Counter()

Counter()

In [4]:
Counter('Darkness')

Counter({'D': 1, 'a': 1, 'r': 1, 'k': 1, 'n': 1, 'e': 1, 's': 2})

In [5]:
Counter({'bacon': 4, 'eggs': 30})

Counter({'bacon': 4, 'eggs': 30})

In [6]:
c = Counter(onions=2, cucumber=19, potato=11, sweet_potato=38)
c

Counter({'onions': 2, 'cucumber': 19, 'potato': 11, 'sweet_potato': 38})

Counts can be retrieved by using keys. If it does not exist in the Counter, count is 0.

In [7]:
c['onions'], c['zombie']

(2, 0)

Counts can be changed. It can be 0 or negative. 

In [8]:
c['sweet_potato'] = 0
'sweet_potato' in c

True

In [9]:
c['sweet_potato'] = -100
'sweet_potato' in c

True

Elements in the Counts can be deleted with del. 

In [10]:
del c['sweet_potato']
'sweet_potato' in c

False

Counter has help methods. <code>elements</code> returns an iterator that generates all the elements.

In [11]:
list(c.elements())

['onions',
 'onions',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato']

Most common items from top to bottom.

In [14]:
c.most_common()

[('cucumber', 19), ('potato', 11), ('onions', 2)]

In [13]:
c.most_common(1)

[('cucumber', 19)]

Counter can subtract others, such as mappings or iterables. It is not an error if some elemtns are missing.

In [17]:
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -6})

In [18]:
# Missing subtract element
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 4, 'b': 0, 'c': -3, 'd': -6})

In [20]:
# Missing original element
c = Counter(a=4, b=2, c=0)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -4})

In [27]:
# Subtracting dictionary
c = Counter(a=4, b=2, c=0, d=-2)
d = {'a':3}
c.subtract(d)
c

Counter({'a': 1, 'b': 2, 'c': 0, 'd': -2})

Counter can also add, but it is called update. Instead of replacing, it simply adds counts.

In [28]:
c.update('abbccc')
c

Counter({'a': 2, 'b': 4, 'c': 3, 'd': -2})

Mathematical operations can be used with Counter objects. Negative counts are removed.

In [45]:
c = Counter(a=2, b=-4, c=0, d=10)
d = Counter(a=-1, b=2, c=3, d=17)

In [47]:
# Add
c + d

Counter({'a': 1, 'c': 3, 'd': 27})

In [48]:
# Subtract
c - d

Counter({'a': 3})

In [49]:
# Intersection
c & d

Counter({'d': 10})

In [50]:
# Union
c | d

Counter({'a': 2, 'b': 2, 'c': 3, 'd': 17})

Unary operator + gets rid of negative and zero elements, and - gets rid of positive and zero elements.

In [33]:
c = Counter(a=2, b=-4, c=0)
+c

Counter({'a': 2})

In [34]:
-c

Counter({'b': 4})

### Defaultdict

Instead of using dictionary's <code>setdefault</code>, </code>defaultdict</code> can achieve higher efficiency and more flexibility. Here, it uses a list for the default value. 

In [38]:
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 11)]
d = defaultdict(list)
d

defaultdict(list, {})

In [16]:
for k, v in l:
    d[k].append(v)

sorted(d.items())

[('bacon', [3, 11]), ('egg', [10]), ('oj', [1, 3])]

Same thing could have achieved using plain dictionary's <code>setdefault</code>.

In [18]:
d2 = {}
for k, v in l:
    d2.setdefault(k, []).append(v)
sorted(d2.items())

[('bacon', [3, 11]), ('egg', [10]), ('oj', [1, 3])]

<code>defaultdict</code> can also use other type for the default value, such as integer. If integer is used, it can be used as a counter.

In [41]:
l = ['bacon', 'egg', 'egg', 'oj', 'bacon', 'bacon', 'bacon', 'bacon']
d = defaultdict(int)
for k in l:
    d[k] += 1
sorted(d.items())

[('bacon', 5), ('egg', 2), ('oj', 1)]

If I need a counter, it is better to use Counter().

In [33]:
d2 = Counter(l)
d2

Counter({'bacon': 5, 'egg': 2, 'oj': 1})

In [35]:
list(d2.elements())

['bacon', 'bacon', 'bacon', 'bacon', 'bacon', 'egg', 'egg', 'oj']

In [36]:
d2.most_common(2)

[('bacon', 5), ('egg', 2)]

Set can also be used as a default type.

In [43]:
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 3)]
d = defaultdict(set)
for k, v in l:
    d[k].add(v)
sorted(d.items())

[('bacon', {3}), ('egg', {10}), ('oj', {1, 3})]

Default value does not have to be set by the type. By passing a value with lambda, any constant value can be set to default.

In [46]:
def constant_factory(value):
    return lambda: value
d = defaultdict(constant_factory('<missing>'))
d.update(name='John', action='ran')
'%(name)s %(action)s to %(object)s' % d

'John ran to <missing>'

In [48]:
d2 = defaultdict(constant_factory(['Must Have!']))
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 11)]
for k,v in l:
    d2[k].append(v)
sorted(d2.items())

[('bacon', ['Must Have!', 3, 10, 1, 3, 11]),
 ('egg', ['Must Have!', 3, 10, 1, 3, 11]),
 ('oj', ['Must Have!', 3, 10, 1, 3, 11])]

### Namedtuple

Making breakfast is tough. Especially when there are many orders. Namedtuple can help with getting the menus straight. Here's a namedtuple regarding what James wants.

In [10]:
Breakfast = namedtuple('Breakfast', 'eggs bacons pancakes oj')
James = Breakfast(3, 2, 3, 1)
James

Breakfast(eggs=3, bacons=2, pancakes=3, oj=1)

In [15]:
James.eggs, James.bacons

(3, 2)

Breakfast menu can also be a string with commans in it.

In [16]:
Breakfast = namedtuple('Breakfast', 'eggs, bacons, pancakes, oj')
James = Breakfast(3, 2, 3, 1)
James

Breakfast(eggs=3, bacons=2, pancakes=3, oj=1)

Or a iterable such as a list.

In [17]:
Breakfast = namedtuple('Breakfast', ['eggs', 'bacons', 'pancakes', 'oj'])
James = Breakfast(3, 2, 3, 1)
James

Breakfast(eggs=3, bacons=2, pancakes=3, oj=1)

With <code>_make</code>, it is possible to use an iterable to create a namedtuple.

In [19]:
quantities = [1, 2, 3, 1]
Mary = Breakfast._make(quantities)
Mary

Breakfast(eggs=1, bacons=2, pancakes=3, oj=1)

With <code>_asdict</code>, namedtuple returns a dictionary.

In [20]:
Mary._asdict()

{'eggs': 1, 'bacons': 2, 'pancakes': 3, 'oj': 1}

With <code>_replace</code>, namedtuple returns a new namedtuple with replaced elements. Mary changed her mind and now wants more eggs.

In [21]:
Mary._replace(eggs=99)

Breakfast(eggs=99, bacons=2, pancakes=3, oj=1)

With <code>_fields</code>, namedtuple can be combined into a new namedtuple. By combining Breakfast and Taste, I made VIP_Breakfast, which has more options, such as temperature, spiciness, and more. 

In [22]:
Breakfast._fields

('eggs', 'bacons', 'pancakes', 'oj')

In [24]:
Taste = namedtuple('Taste', 'temperature, spiciness, sweetness, savoriness')
VIP_Breakfast = namedtuple('VIP_Breakfast', Breakfast._fields + Taste._fields)
Nom = VIP_Breakfast(1, 3, 5, 7, 'hot', 'spicy', 'not sweet', 'very savory')
Nom

VIP_Breakfast(eggs=1, bacons=3, pancakes=5, oj=7, temperature='hot', spiciness='spicy', sweetness='not sweet', savoriness='very savory')

Passing defaults when initiating namedtuple sets defaults values starting from farthest right parameters. By providing defaults with an array length of four, default values are set for Taste. It is possible to override default values by providing right positional argument or keyword argument.

In [32]:
Taste = namedtuple('Taste', 'temperature, spiciness, sweetness, savoriness')
VIP_Breakfast = namedtuple('VIP_Breakfast', 
                           Breakfast._fields + Taste._fields,
                           defaults=['hot', 'regular', 'semi-sweet', 'savory'])
Nom = VIP_Breakfast(1, 3, 5, 7, 'cold', savoriness='not savory')
Nom

VIP_Breakfast(eggs=1, bacons=3, pancakes=5, oj=7, temperature='cold', spiciness='regular', sweetness='semi-sweet', savoriness='not savory')

Using <code>_field_defaults</code>, it is easy to check the default values.

In [31]:
Nom._field_defaults

{'temperature': 'hot',
 'spiciness': 'regular',
 'sweetness': 'semi-sweet',
 'savoriness': 'savory'}

Using getattr is another way to get the field of namedtuple.

In [33]:
getattr(Nom, 'oj')

7

If a server wrote the order with a dictionary format, it is no problem. Namedtuple can be initialized with double stars.

In [36]:
VIP_order = {'eggs': 2, 'bacons': 3, 'pancakes': 9, 'oj': 1}
Nom = VIP_Breakfast(**VIP_order)
Nom

VIP_Breakfast(eggs=2, bacons=3, pancakes=9, oj=1, temperature='hot', spiciness='regular', sweetness='semi-sweet', savoriness='savory')

### Deque

Pronounced as "deck," deque is used to work on only certain number of elements. For example, it is used for saving last five visits in history or saving last couple lines of a file. Another usage is inserting and popping elements in the beginning of the deque. 

In [27]:
d = deque('abc', 3)
d

deque(['a', 'b', 'c'])

In [28]:
d.append('d')
d

deque(['b', 'c', 'd'])

In [29]:
d.appendleft('1')
d

deque(['1', 'b', 'c'])

In [30]:
d.extend(['d', 'e'])
d

deque(['c', 'd', 'e'])

In [31]:
# Extendleft's arguments are inserted in a reversed order.
d.extendleft([0, 1])
d

deque([1, 0, 'c'])

In [32]:
d.index(0)

1

In [33]:
d.pop()

'c'

In [34]:
d.popleft()

1

In [35]:
d

deque([0])

In [36]:
d.remove(0)
d

deque([])

In [38]:
d.extend([0,1,2])
d

deque([0, 1, 2])

In [41]:
d.reverse()
d

deque([2, 1, 0])

In [43]:
# cannot insert when it is full
d.insert(1, 7)
d

IndexError: deque already at its maximum size

In [47]:
d.rotate()
d

deque([0, 2, 1])

In [49]:
d.rotate(-1)
d

deque([1, 0, 2])

In [50]:
d.maxlen

3

In [51]:
d.clear()
d

deque([])

It is very similar to list, except that it has a fixed size, and it can accept elements in the beginning efficiently.

In [52]:
# Just more exmaples.
# last 3 numbers of data can be achieved like this
for i in range(10):
    d.append(i)
d

deque([7, 8, 9])

Deque can also be used for grabbing last 5 lines of file or getting moving averages. 

## Itertools

### Permutations

Permutations function simply return permutations. It can be used for cases where order matters, and there is no overlap. For example, making a sequence of music notes with given possible notes without overlap is an example of using permutations.

In [249]:
list(permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

Here is my implementation that returns a list. However, itertools version returns a generator. 

In [240]:
def our_permutations(iterable, num=None):
    """Return a list of permutations."""
    length = len(iterable)
    num = length if num is None else num
    if num > length:
        return []
    elif num == 0:
        return [()]
    else:
        result = []
        for i in range(length):
            rest = tuple(our_permutations(iterable[0:i] + iterable[i+1:], num - 1))
            for j in range(len(rest)):
                result.append(tuple(iterable[i]) + rest[j])
        return result

In [243]:
our_permutations('BC', 1)

[('B',), ('C',)]

In [244]:
our_permutations('ABCD')

[('A', 'B', 'C', 'D'),
 ('A', 'B', 'D', 'C'),
 ('A', 'C', 'B', 'D'),
 ('A', 'C', 'D', 'B'),
 ('A', 'D', 'B', 'C'),
 ('A', 'D', 'C', 'B'),
 ('B', 'A', 'C', 'D'),
 ('B', 'A', 'D', 'C'),
 ('B', 'C', 'A', 'D'),
 ('B', 'C', 'D', 'A'),
 ('B', 'D', 'A', 'C'),
 ('B', 'D', 'C', 'A'),
 ('C', 'A', 'B', 'D'),
 ('C', 'A', 'D', 'B'),
 ('C', 'B', 'A', 'D'),
 ('C', 'B', 'D', 'A'),
 ('C', 'D', 'A', 'B'),
 ('C', 'D', 'B', 'A'),
 ('D', 'A', 'B', 'C'),
 ('D', 'A', 'C', 'B'),
 ('D', 'B', 'A', 'C'),
 ('D', 'B', 'C', 'A'),
 ('D', 'C', 'A', 'B'),
 ('D', 'C', 'B', 'A')]

In [205]:
our_permutations('ABCD', 2)

[('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'A'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('D', 'C')]

And this is from python documentation. 

In [171]:
def permutations(iterable, r=None):
    # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable) # ('A', 'B', 'C', 'D')
    n = len(pool) # 4
    r = n if r is None else r # 2
    if r > n:
        return
    indices = list(range(n)) # [0, 1, 2, 3]
    cycles = list(range(n, n-r, -1)) # [1, 0]
    yield tuple(pool[i] for i in indices[:r]) # tuple(pool[i] for i in [0, 1]) -> ('A', 'B')
    while n: # 4
        for i in reversed(range(r)): # i in reversed(range(2)) -> 1, 0
            cycles[i] -= 1 # -1, 0
            if cycles[i] == 0: 
                indices[i:] = indices[i+1:] + indices[i:i+1] 
                # indices[1:] = indices[2:] + indices[1:2]
                # [1,2,3] = [2,3] + [1]
                cycles[i] = n - i # cycles[0] = 4
            else: # i: 1
                j = cycles[i] # j: -1
                indices[i], indices[-j] = indices[-j], indices[i] # 1, 1 = 1, 1
                yield tuple(pool[i] for i in indices[:r]) 
                # tuple(pool[i] for i in [0, 1]) -> ('A', 'B')
                break
        else:
            return

In [172]:
permutations('ABC', 5)

<generator object permutations at 0x7f31ab408350>

### Combinations

Similar to permutations, it returns combinations. 

In [44]:
list(combinations('ABCD', 3))

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]

Here is combinations that I wrote similar to our_permutations. 

In [40]:
def our_combinations(iterable, num=None):
    """Return a list of combinations."""
    length = len(iterable)
    num = length if num is None else num
    if num > length:
        return []
    elif num == 0:
        return [()]
    else:
        result = []
        for i in range(length - num + 1):
            rest = tuple(our_combinations(iterable[i+1:], num - 1))
                result.append(tuple(iterable[i]) + rest[j])
        return result

In [45]:
our_combinations('ABCD', 3)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]

In python <a href="https://docs.python.org/3/library/itertools.html#itertools.combinations">documentation</a>, combinations is written in terms of permutations. It uses permutations to create all the possible indices of making permutations. Then it checks whether the set of indices matches the sorted indices. Although it is an interesting way to implement combinations, it is not very efficient. 

In [ ]:
def p_combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in permutations(range(n), r):
        if sorted(indices) == list(indices):
            yield tuple(pool[i] for i in indices)

### Chain

Chain is used to put multiple iterables together. 

In [3]:
l1 = ['a', 'b', 'c']
l2 = ['d', 'e', 'f']
for element in chain(l1, l2):
    print(element)

a
b
c
d
e
f


I wonder whether using chain is faster than just using + with lists.

In [4]:
long_lst = list(range(10000))
long_lst2 = list(range(10000, 20000))

In [5]:
%timeit [x for x in long_lst+long_lst2]

830 µs ± 4.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
%timeit [x for x in chain(long_lst, long_lst2)]

792 µs ± 7.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


So, using chain is slighly faster than simply using +. How about using more + vs. chain?

In [7]:
%timeit [x for x in long_lst+long_lst2+long_lst+long_lst2]

2.16 ms ± 45 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit [x for x in chain(long_lst,long_lst2,long_lst,long_lst2)]

1.59 ms ± 6.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Using more chain is more efficient than +.

### Count

Even though it is count, it is imported as count_from here. It simply counts from a given input. 

In [13]:
list(zip(['a', 'b', 'c', 'd'], count_from(1)))

[('a', 1), ('b', 2), ('c', 3), ('d', 4)]

In [2]:
list(zip(['a', 'c', 'e', 'g'], count_from(1, 2)))

[('a', 1), ('c', 3), ('e', 5), ('g', 7)]

Different from <code>range</code>, count_from does not take the end limit. 

### 